In [1]:
import mindspore
import argparse
import numpy as np
import logging
import mindspore.dataset as ds
import os

import json

from tqdm import tqdm
from datetime import datetime
from mindspore.nn import CrossEntropyLoss
from mindspore import nn, ops
from mindspore.train.serialization import save_checkpoint
from mindnlp.transforms import BertTokenizer
from mindnlp.modules import Accumulator

[ERROR] ME(49483:139991715923776,MainProcess):2023-05-10-09:02:28.680.215 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcu*.so need by mindspore-gpu) is not found. Please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches. Please refer to the installation guidelines: https://www.mindspore.cn/install
[ERROR] ME(49483:139991715923776,MainProcess):2023-05-10-09:02:28.703.582 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcudnn*.so need by mindspore-gpu) is not found. Please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches. Please refer to the installation guidelines: https://www.mindspore.cn/install
/data/miniconda3/envs/mindspore/lib/python3.7/site-packages/mindnlp/utils/download.py:26: Tqd

In [41]:
from mindnlp.utils import cache_file

url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path, _ = cache_file('train_with_summ.txt', './', url)

In [43]:
from mindspore.dataset import TextFileDataset

dataset = TextFileDataset(str(path), shuffle=False)

In [44]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer.sep_token_id

102

In [45]:
import numpy as np

def read_map(text):
    data = json.loads(text.tobytes())
    return np.array(data['article']), np.array(data['summarization'])

dataset = dataset.map(read_map, 'text', ['article', 'summary'])
dataset = dataset.map(tokenizer, 'article')
dataset = dataset.map(tokenizer, 'summary')

In [3]:
model_config = {
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_embd": 768,
  "n_head": 12,
  "n_ctx": 1024,
  "n_layer": 10,
  "num_hidden_layers": 10,
  "top_k": 50,
  "top_p": 1.0,
  "num_return_sequences": 1,
  "n_positions": 1024,
  "vocab_size": 13317
}

In [ ]:
epochs = 6
batch_size = 8

lr = 1e-4
warmup_steps = 2000
grad_accumulation = 2
max_grad_norm = 1.0

In [ ]:
def create_model(args, vocab_size):
    """
    :param args:
    :param vocab_size:字典大小
    :return:
    """
    if args.pretrained_model:  # 如果指定了预训练的GPT2模型
        model = gpt2.GPT2LMHeadModel.load(args.pretrained_model)
        # model = gpt2.GPT2LMHeadModel.load(args.pretrained_model)
    else:  # 若没有指定预训练模型，则初始化模型
        # model_config = config_gpt2.GPT2Config.from_json_file(args.model_config)
        model_config = config_gpt2.GPT2Config.from_json(args.model_config)
        model = gpt2.GPT2LMHeadModel(config=model_config)
    # 根据tokenizer的vocabulary调整GPT2模型的voca的大小
    model.resize_token_embeddings(vocab_size)
    logger.info('model config:\n{}'.format(model.config.to_json_string()))
    return model, model.config.to_dict().get("n_ctx")

In [ ]:
def calculate_loss_and_accuracy(outputs, labels):
    """
    计算非pad_id的平均loss和准确率
    :param outputs:
    :param labels:
    :param device:
    :return:
    """
    logits = outputs[0]  # 每个token用来预测下一个token的prediction_score,维度:[batch_size,token_len,voca_size]
    # 用前n-1个token，预测出第n个token
    # 用第i个token的prediction_score用来预测第i+1个token。
    # 假定有input有n个token，则shift_logits表示model中第[0,n-2]个token的prediction_score，shift_labels表示第[1，n-1]的label
    shift_logits = logits[..., :-1, :]
    shift_labels = labels[..., 1:]
    shift_labels = shift_labels.astype(mindspore.int32)

    loss_fct = CrossEntropyLoss(ignore_index=pad_id, reduction='sum')  # 忽略pad_id的loss,并对所有的非pad_id的loss进行求和
    loss = loss_fct(shift_logits.view(-1, shift_logits.shape[-1]),
                    shift_labels.view(-1))
    
    preds = shift_logits.argmax(axis=-1) # preds表示对应的prediction_score预测出的token在voca中的id。维度为[batch_size,token_len]

    # 对非pad_id的token的loss进行求平均，且计算出预测的准确率
    # not_ignore = shift_labels != pad_id
    not_ignore = shift_labels.ne(pad_id)  # 进行非运算，返回一个tensor，若targets_view的第i个位置为pad_id，则置为0，否则为1
    num_targets = not_ignore.astype(mindspore.int64).sum()  # 计算target中的非pad_id的数量

    correct = (shift_labels == preds) & not_ignore  # 计算model预测正确的token的个数，排除pad的tokne
    correct = correct.float().sum()

    accuracy = correct / num_targets
    loss = loss / num_targets
    return loss, accuracy


In [ ]:
# Define forward function
def forward_fn(data, label):
    """_summary_ 前向推理步骤

    Args:
        data (_type_): _description_
        label (_type_): _description_

    Returns:
        _type_: _description_
    """
    logits = model(data)
    loss, accuracy = calculate_loss_and_accuracy(logits, label)
    return loss / accumulate_step


# Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())


# Define function of one-step training
# @mindspore.jit
def train_step(data, label):
    """_summary_ 训练步骤

    Args:
        data (_type_): _description_
        label (_type_): _description_

    Returns:
        _type_: _description_
    """
    loss, grads = grad_fn(data, label)
    loss = ops.depend(loss, accumulator(grads))
    return loss

以下两个函数用于对原始语料进行处理的。
第一个preprocess_raw_data函数是根据获取到的文本内容，将其转换为相应的token id。
第二个preprocess_mmi_raw_data函数是训练MMI模型，转换的形式与第一个相似。

In [ ]:
# 记录模型参数数量
num_parameters = 0
parameters = model.trainable_params()
for parameter in parameters:
    num_parameters += parameter.numel()
logger.info('number of model parameters: {}'.format(num_parameters))

保存模型参数，并对wte进行处理成能识别的训练参数

In [ ]:
def ckpt_to_mindspore(mth_file, size:str=None):
    """_summary_ 
    Resolve the parameter wte.embedding of the generative model,
    the lack of training transformer prefix in table allows for loading

    Args:
        mth_file (_type_): _description_
        size (str, optional): _description_. Defaults to None.

    Raises:
        ImportError: _description_
        RuntimeError: _description_

    Returns:
        _type_: _description_
    """
    try:
        import mindspore
    except:
        raise ImportError(f"'import mindspore' failed, please install mindspore by "
                          f"`pip mindspore torch` or instructions from 'https://www.mindspore.cn/install'")

    size = "mindspore" if not size else size # rename ckpt

    from mindspore import Tensor
    from mindspore.train.serialization import save_checkpoint

    logging.info('Starting checkpoint conversion.')
    ms_ckpt = []
    state_dict = mindspore.load_checkpoint(mth_file)

    for k, v in state_dict.items():
        if 'wte.embedding_table' in k:
            k = k.replace('wte.embedding_table', 'transformer.wte.embedding_table')
        ms_ckpt.append({'name': k, 'data': Tensor(v.numpy())})

    try:
        save_checkpoint(ms_ckpt, mth_file)
    except:
        raise RuntimeError(f'Save checkpoint to {mth_file} failed, please checkout the path.')

    return mth_file

In [ ]:
def train(model, train_list, multi_gpu, args):
    """_summary_ 训练逻辑，进行数据加载之后的推理和保存模型

    Args:
        model (_type_): _description_
        train_list (_type_): _description_
        multi_gpu (_type_): _description_
        args (_type_): _description_

    Raises:
        exception: _description_
    """
    train_dataset = MyDataset(train_list)
    train_dataloader = ds.GeneratorDataset(train_dataset, column_names="input_ids" ,num_parallel_workers=args.num_workers, shuffle=True)
    train_dataloader = train_dataloader.padded_batch(batch_size=args.batch_size, drop_remainder=True, pad_info={})
    train_dataloader = train_dataloader.repeat(1)
    train_dataloader = train_dataloader.shuffle(10)
    
    # 计算所有epoch进行参数优化的总步数total_steps
    total_steps = int(train_dataset.__len__() * args.epochs / args.batch_size / args.gradient_accumulation)
    logger.info('total training steps = {}'.format(total_steps))

    logger.info('starting training')
    # 记录 out of memory的次数
    oom_time = 0
    # 开始训练
    for epoch in range(args.epochs):
        epoch_start_time = datetime.now()
        size = len(train_dataloader)
        # batch_idx为int，input_ids为一个tensor
        for batch_idx, input_ids in enumerate(train_dataloader.create_tuple_iterator()):
            # 注意：GPT2模型的construct()函数，是对于给定的context，生成一个token，而不是生成一串token
            # GPT2Model的输入为n个token_id时，输出也是n个hidden_state，使用第n个hidden_state预测第n+1个token
            # 解决在运行过程中，由于显存不足产生的cuda out of memory的问题
            try:
                input_id = input_ids[0].astype(mindspore.int64)
                loss = train_step(input_id, input_id)
                
                if batch_idx % args.log_step == 0:
                    loss, current = loss.asnumpy(), batch_idx
                    logger.info(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")
            except RuntimeError as exception:
                if "out of memory" in str(exception):
                    oom_time += 1
                    logger.info("WARNING: ran out of memory,times: {}".format(oom_time))    
                else:
                    logger.info(str(exception))
                    raise exception
        logger.info('saving model for epoch {}'.format(epoch + 1))
        if args.train_mmi:  # 当前训练MMI模型
            model_path = join(args.mmi_model_output_path, 'model_epoch{}'.format(epoch + 1))
        else:  # 当前训练对话模型
            model_path = join(args.dialogue_model_output_path, 'model_epoch{}'.format(epoch + 1))
        if not os.path.exists(model_path):
            os.mkdir(model_path)
        model_to_save = model.module if hasattr(model, 'module') else model
        model_path = f"{model_path}/mindspore_model.ckpt"
        save_checkpoint(model_to_save, model_path)
        ckpt_to_mindspore(model_path)
        logger.info('epoch {} finished'.format(epoch + 1))
        epoch_finish_time = datetime.now()
        logger.info('time for one epoch: {}'.format(epoch_finish_time - epoch_start_time))
    logger.info('training finished')

In [ ]:
# 加载数据
logger.info("loading traing data")
if args.train_mmi:  # 如果是训练MMI模型
    with open(args.train_mmi_tokenized_path, "r", encoding="utf8") as f:
        data = f.read()
else:  # 如果是训练对话生成模型
    with open(args.train_tokenized_path, "r", encoding="utf8") as f:
        data = f.read()
data_list = [line.rstrip() for line in data.split("\n") if line.rstrip()]
train_list = data_list

In [ ]:
# 开始训练
train(model, train_list, multi_gpu, args)